# <img src="./images/SQLIcon.png?modified=23223" width=80px height=80px style="vertical-align: middle;"> SQL `JOIN` Types


In [ ]:
#@title ### Run the following cell to download the necessary files for this lesson { display-mode: "form" } 
#@markdown Don't worry about what's in this collapsed cell

print('Downloading video_player.py...')
!wget https://s3-eu-west-1.amazonaws.com/aicore-portal-public-prod-307050600709/lesson_files/492b5fbe-c793-4bc1-88a0-0244223a26db/video_player.py -q -O video_player.py
import video_player
print('Downloading staff_db.tar...')
!wget https://s3-eu-west-1.amazonaws.com/aicore-portal-public-prod-307050600709/lesson_files/492b5fbe-c793-4bc1-88a0-0244223a26db/staff_db.tar -q -O staff_db.tar
print('Downloading install_staff_db.mp4...')
!wget https://s3-eu-west-1.amazonaws.com/aicore-portal-public-prod-307050600709/lesson_files/492b5fbe-c793-4bc1-88a0-0244223a26db/install_staff_db.mp4 -q -O install_staff_db.mp4


There are many different joins you can perform in SQL, depending on the requirements needed in the resultant dataset. SQL joins are often represented by a *Venn diagram*. A SQL **Venn diagram** is a graphical representation of the relationships between tables. It is a type of Venn diagram that is used specifically to show the relationships between tables and the data they contain.

In a SQL Venn diagram, each circle represents a table in the database, and the areas of overlap between the circles represent the overlapping data between the tables. The coloured sections represent the data which will retrieved by the specific join. In these examples the Venn diagram will be shown along with the join to help you understand the data which will retrieved by the join. 

>All example queries in the this notebook were run using the example `staff_db` database provided on google collab. You will need to download the database from collab and then restore it with PgAdmin4 so you can run the queries. Please watch the video below to see an example of how to do this.

<h3 style="color: rgb(241, 90, 36)">Watch it in action</h3>


In [ ]:
#@title ### Run the cell to play the video{ display-mode: "form" } 
video_player.play_video("install_staff_db.mp4")

#### `SELF JOIN`

<img src="images/self_join.png?modified=3242323423">

A `SELF JOIN` is joining a table to itself. This can be a useful `JOIN` if the table references itself in some way.

In the `current_employees` table of the `staff` database, each employee has an `integer` in the `manager_id` column which represents the `employee_id` of their manager. If we wanted data which showed the name of the manager for each employee, we could do so with the following `SELF JOIN`: 

In [ ]:
SELECT employees.employee_id,
       employees.name,
       employees.manager_id,
       managers.name AS manager_name
FROM
    current_employees employees
INNER JOIN 
    current_employees managers ON employees.manager_id = managers.employee_id;

Here the query has to reference the `current_employees` table twice since we're joining the table to itself. We have aliased the first instance of the table as `employees` and the second instance of the table as `managers`. We then join the tables where the `manager_id` is equal to the `employee_id`, producing rows matching the manager to the respective employee. Notice `Tim` doesn't appear in the data since they don't have a `manager_id` (looks like Tim is the boss). 

#### `JOIN/INNER JOIN`

<img src='images/Inner_join.png?modified=231321'>

The `INNER JOIN` checks two or more tables and if the condition matches in the **join predicate** then the rows are returned in the resultant dataset. This will find all rows in `Table A` which matches rows in `Table B` based on a condition. In mathematical terms if you've studied set theory this would be the *intersection* of the two tables. You can see the **intersection** of the data as the orange section in the diagram. 

Let's use the `INNER JOIN` to find the salaries of current employees:

In [ ]:
SELECT managers.employee_id,
       managers.name,
       managers.salary
FROM
    current_employees
INNER JOIN 
    managers ON managers.employee_id = current_employees.employee_id;

<img src='images/inner_join_query.png?modified=3423432'>

In the **join predicate** we are matching the `employee_id` columns in the `managers` and `current_employees` table, `managers.employee_id = current_employees.employee_id`. Which means if an `employee_id` exists in both tables then the rows are returned. You can see in the `managers` table there are more `managers` than managers that exist in the `current_employees` table. So not every manager will exist in the resultant dataset, only the managers which are current employees.

#### `LEFT JOIN/LEFT OUTER JOIN`

<img src='images/Left_join.png?modified=22331321'>

The `LEFT JOIN` or `LEFT OUTER JOIN` joins all rows from the left table and matchings rows from the right table based on the **join predicate**. When joined, all data from the left table and only data which **intersects** (exists in) the right table is returned. 

Let's say we wanted to look at the `current_employees` table and check which rows contain data for managers. We know we want to keep all data in the table `current_employees` so this should be the table on the **left** and the `managers` table on the **right**. So how do we state which table is the left table and which is the right? 

>The **left** table will always be the table the data is being joined onto, the table specified in the `FROM` statement. The **right** table will be the table which is being joined onto the existing set of data. You might think it would be the order the tables are specified in the join predicate. This wouldn't be valid since `a = b` implies `b = a`. 

We can check all managers which are currently employees using the following query:


In [ ]:
SELECT current_employees.employee_id,
       current_employees.name,
       managers.salary,
       current_employees.manager_id
FROM
    current_employees
LEFT OUTER JOIN 
    managers ON current_employees.employee_id = managers.employee_id;

<img src='images/left_join_query.png?modified=32434324'>

Notice the rows for the `salary` column have some `NULL` values, since the `salary` values come from the `managers` table. We are keeping all data in the `current_employees` table. We can't get the value of `salary` for staff which aren't managers since this data comes from the `managers` table. The two tables don't intersect on the `salary` column which belongs to the `non-managers` table. 

`LEFT JOINS` and `OUTER` joins are used when you know you want to keep the majority of the data in one table. Why the majority of the data? Sometimes you might want to keep all data in the left table that doesn't exist in the right table. This is known as a *left outer exclusive join*, since it's excluding the intersection of the two tables. Conversely you can consider a `LEFT OUTER JOIN` as a *left outer inclusive join*. 

#### Exclusive `LEFT JOIN`

<img src='images/exclusive_left_join.png?modified=24324'>

In the image above, we are getting all the records in the left table minus the records in the right table i.e removing the intersection of the two tables. The intersection of the two tables will be the managers from the `managers` table since this is the table on the right. Following on from our previous query we just need to add a `WHERE` clause to the statement for SQL to perform the **exclusive** `JOIN`:

In [ ]:
SELECT current_employees.employee_id,
       current_employees.name,
       managers.salary,
       current_employees.manager_id
FROM
    current_employees
LEFT OUTER JOIN 
    managers ON current_employees.employee_id = managers.employee_id
WHERE
    managers.employee_id is NULL;

The **intersection** was removed with the `WHERE managers.employee_id is NULL` statement. Remember the `managers.employee_id` column is still is available to be filtered with the `WHERE` statement. Though not viewable in the final data since it is not selected in the `SELECT` statement, as `SELECT` comes after `WHERE` in the order of execution. `WHERE managers.employee_id is NULL` can only be the case when the current employee isn't a manager. This can more clearly seen if the column `managers.employee_id` is added to the `SELECT` statement of the first query:

In [ ]:
SELECT current_employees.employee_id,
       current_employees.name,
       managers.salary,
       current_employees.manager_id,
       managers.employee_id AS managers_table_id
FROM
    current_employees
LEFT OUTER JOIN 
    managers ON current_employees.employee_id = managers.employee_id;

<img src='images/exclusive_left_join_results.png?modified=342324'>

The results shows, employees who aren't managers have a `NULL` value for the column `manages_table_id` since they don't exist there. This allows us to complete the exclusive join. 

>An easier way to remember the **exclusive left join** is, filter with `WHERE` by the right table and the key you're joining on, in this case the `employee_id` key. 


#### `RIGHT JOIN/RIGHT OUTER JOIN`

<img src='images/right_join.png?modified=231321'>

The `RIGHT JOIN` or `RIGHT OUTER JOIN` joins all rows from the right table and matchings rows from the left table based on the join condition. When joined, all data from the **right table** and only the data in the **left table** which exists in the right table is returned. 

The `RIGHT JOIN` is performed similarly to the `LEFT JOIN`, but you want to keep all records in the table being joined(table on the right), not the table in the `FROM` statement. 

>It's important to note that that a `LEFT JOIN` is interchangeable with a `RIGHT JOIN`, you can just change the order of the tables and the side to join:


In [ ]:
SELECT current_employees.employee_id,
       current_employees.name,
       managers.salary,
       current_employees.manager_id,
FROM
    current_employees
LEFT OUTER JOIN 
    managers ON current_employees.employee_id = managers.employee_id;

Returns the same data as:

In [ ]:
SELECT current_employees.employee_id,
       current_employees.name,
       managers.salary,
       current_employees.manager_id,
FROM
    managers
RIGHT OUTER JOIN 
    current_employees ON current_employees.employee_id = managers.employee_id;

You'll find that most people will prefer using `LEFT JOIN`'s over `RIGHT JOINS`'s. There's the theory this is due to most languages being read from left-to-right so it's more natural for most people to think this way. You'll rarely see a `RIGHT JOIN` used in SQL since you can always choose the table to join onto to be on the **left** instead of right. 

#### `Exclusive RIGHT JOIN`

<img src='images/exclusive_right_join.png?modified=342432'>

The exclusive `RIGHT JOIN` can be written similarly to the exclusive `LEFT JOIN`:

In [ ]:
SELECT current_employees.employee_id,
       current_employees.name,
       managers.salary,
       current_employees.manager_id
FROM
    managers
RIGHT OUTER JOIN 
    current_employees ON current_employees.employee_id =  managers.employee_id
WHERE
    managers.employee_id is NULL;

Here we still need to filter using the `WHERE` clause: where the `managers` table `employee_id` is `NULL`. Since we want to remove all records which don't exist in the `managers` table (the table on the **left**).

#### `FULL OUTER JOIN`

<img src='images/Full_outer.png?modified=22331321'>

A `FULL OUTER JOIN`, also sometimes know as a `FULL JOIN`, returns all the rows from both the **right** and the **left** tables along with any **matching** rows in both tables. If the two tables have no rows in common, the `JOIN` will still return all rows from both tables, but with `NULL` values that correspond to **missing** data. 

This is also an uncommonly used `JOIN` but can be useful in a few cases:

- **Comparing data between tables**: If you have two tables and want to check their differences, you can use the `FULL OUTER JOIN.` The `NULL`'s in the result will represent the difference between the two tables. 
- **Merging two tables together**: If you wanted to `JOIN` two tables together, but wanted to make sure there was no missing data. Then you could perform the `FULL OUTER JOIN` and check what data needs to be inserted to complete the new table. The missing data will be indicated by the `NULL` values. 

Say we wanted to merge the two tables, `non_managers` and `managers` to create a new table `employees` containing all employees in the company. We could perform a `FULL OUTER JOIN` first, to check what data would be missing if we needed to merge these tables:

In [ ]:
SELECT non_managers.employee_id AS non_manager_employee_id,
       non_managers.name AS name_of_non_manager,
       managers.salary,
       managers.name AS name_of_manager,
       managers.employee_id AS manager_employee_id     
FROM
    non_managers
FULL OUTER JOIN
    managers ON managers.employee_id = non_managers.employee_id;

<img src='images/full_outer_join_results.png?modified=231231'>

Looking at the results, it seems that every row has an `employee_id` and `name` of staff member. For every manager, we have a value for `salary` but for non managers we don't. We would need to be aware about inserting a `salary` value for each member of staff who are not managers, if we wanted to create a new table derived from this data.  

#### `CROSS JOIN`

<img src='images/cross_join.png?modified=231321'>

For every record in **table A** the `CROSS JOIN` will match every record in **table B**. This returns the *Cartesian product* of the rows from the tables. In Mathematics the **Cartesian product(denoted A x B)**, is the set of all *ordered pairs(a,b)*, where **a** exists in the set **A** and **b** exists in the set **B**. This `JOIN` can be really useful if you want to find all **combinations** of values between two tables, which the **Cartesian product** returns. Additionally useful if you want to generate test data, it gets all possible **combinations** between tables which you can use as test data. Just be careful with `CROSS JOIN`S, they will generated a large amount of data.

For instance, say you wanted to find all possible combinations of managers each non manager can have between the `non_managers` and `managers` tables:

In [ ]:
SELECT managers.name AS managers_name,
       non_managers.name AS staff_name
FROM
    managers
CROSS JOIN
    non_managers;

The `CROSS JOIN` this will generate all possible **combinations** of **managers** and **staff**. You can see the `CROSS JOIN` has much simpler syntax: you just need to specify which tables you want to `JOIN` and then `SELECT` the columns you want in the resultant data. 

<h3 style="color: rgb(241, 90, 36)">Which JOIN to choose?</h3>

When first using **joins** it can be very confusing just what kind of join you need to use. Here are some tips which might help when you need to decide:

- If you are referencing the same table you will need a `SELF JOIN`
- You can perform everything with a `LEFT JOIN` you can with a `RIGHT JOIN`. Pick the one that feels most comfortable to use to you.
- Think of the size of the tables, when performing a `LEFT JOIN` the resulting set will be the size of whatever table you `SELECT` on the **left**(the table in the `FROM` statement). If for instance you had an `orders` table of size **m** and a `products` table you were joining of size **n** much smaller than **m**. Then you can be sure you want the `orders` table on the left. 
- Do I want to check the differences between two tables? Then maybe a `FULL OUTER JOIN` would be useful here. 
- Checking where tables match? Then you would be looking to use an `INNER JOIN`. 
- Drawing a **Venn diagram** for the tables you want to join can help provide clarity on which type of `JOIN` you require. 

## Key Takeaways

- Not all SQL `JOIN`s are created equal, by far the most useful/used are the `INNER JOIN` and `LEFT JOIN`
- Remember the "table on the left" is the table being joined on to. The "table on the right" is the table being joined
- You almost never need to use a `RIGHT JOIN` as the same query can almost always be performed with a `LEFT JOIN`
- Be careful when using `CROSS JOIN`'s as they can generate a **lot** of data. They're great to get combinations of values though.
- You may need to perform multiple `JOIN`s in a query 